In [1]:
import re
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
import pickle
import os

In [3]:
def get_var_name(var_value, namespace):
    """
    Given a variable value and namespace (e.g. globals() or locals()),
    returns the name of the variable as a string.
    """
    for name in namespace:
        if namespace[name] is var_value:
            return name
    return None

In [4]:
path = 'driver/chromedriver.exe'
driver = webdriver.Chrome(path)

In [5]:
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

In [6]:
driver.get('https://www.linkedin.com/login');
time.sleep(2)

In [7]:
'''Login Credentials'''
username = 'hudafebrianto12@gmail.com'
password = 'Vancha12'

Login

In [8]:
driver.find_element_by_xpath('//*[@id="username"]').send_keys(username)
driver.find_element_by_xpath('//*[@id="password"]').send_keys(password)

Submit

In [9]:
driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button').click()
# driver.implicitly_wait(30)

Verification

In [12]:
# '''verification via email'''
# driver.find_element_by_xpath('//*[@id="input__email_verification_pin"]').send_keys(361673)
# driver.find_element_by_xpath('//*[@id="email-pin-submit-button"]').click()

In [13]:
# '''skip account data verification'''
# driver.find_element_by_xpath('//*[@id="ember20"]/button[2]').click()

Search Bar

In [37]:
job_name = 'maintenance electrical'

In [38]:
job_title = '%20'.join(job_name.split())
driver.get(f'https://www.linkedin.com/jobs/search/?keywords={job_title}&location=Indonesia&refresh=true')
driver.implicitly_wait(20)

In [39]:
job_results = []

try:
    pages_block = driver.find_elements_by_class_name('artdeco-pagination__indicator')
    driver.implicitly_wait(1)
    
    total_pages = pages_block[-1].text
    total_pages = int(total_pages)
except Exception as e:
    total_pages = 1

total_data = driver.find_element_by_class_name('jobs-search-results-list__subtitle').text.split()[0]

print(f'Total {total_data} data from {total_pages} total pages.')

Total 260 data from 11 total pages.


In [40]:
job_num = 1

for i in range(1, total_pages + 1):
    if total_pages != 1:
        pages_block = driver.find_elements_by_class_name('artdeco-pagination__indicator')
        
        all_pages_number = [page.text for page in pages_block]

        if i == total_pages:
            print('Last page', i, 'from', total_pages, 'total pages.')
            break

        page_block = pages_block[all_pages_number.index(str(i))]
        
        page_block.click()
        driver.implicitly_wait(3)


    '''scraping code'''
    jobs_block = driver.find_elements_by_class_name('jobs-search-results__list-item')

    for job in jobs_block:
        if len(job_results)%15 == 0:
            driver.execute_script('arguments[0].scrollIntoView();', job)
            driver.implicitly_wait(3)
        
        try:
            job.click()
            driver.implicitly_wait(3)
        except Exception as e:
            continue

        # job metadata
        try:
            metadata = job.find_element_by_class_name('artdeco-entity-lockup__content').text.split('\n')
            driver.implicitly_wait(3)

        except Exception as e:
            print('❌ Failed when try to scrape metadata ...')
            print(e)
            metadata = [None, None, None, None]

        job_information = dict(zip(
            ['Title', 'Company', 'Area', 'Type'],
            metadata
        ))
        
        # job skills
        if len(driver.find_elements_by_class_name('jobs-unified-top-card__job-insight')) < 4:
            skills = None
        else:
            try:
                driver.find_element_by_class_name('jobs-unified-top-card__job-insight-text-button').click()
                driver.implicitly_wait(1)

                try:
                    skills = driver.find_element_by_class_name('job-details-skill-match-status-list').text.split('\n')
                    skills = [skill for skill in skills if skill != 'Tambahkan']
                    
                except Exception as e:
                    print('❌ Failed when try to scrape skills ...')
                    print(e)
                    skills = None

                driver.find_element_by_class_name('artdeco-button__icon').click()
            except Exception as e:
                print('❌ Failed when try to click skills block ...')
                print(e)
                skills = None

        job_information['Skills'] = skills

        # job description
        try:
            description = driver.find_element_by_class_name('jobs-description__container').text
            description = re.sub('\n', ' ', description)

        except Exception as e:
            print('❌ Failed when try to scrape description ...')
            print(e)
            description = None

        job_information['Description'] = description


        job_results.append(job_information)


        print(f'[{job_num}, {i}, {total_data}] {job_information["Title"]} 👍')
        job_num += 1


    '''go to the next page'''
    if total_pages != 1:
        next_page = pages_block[all_pages_number.index(str(i)) + 1]
        print(f'📖 Will move to the next page ({i} -> {next_page.text}) from {total_pages} total pages ...')
        next_page.click()
        driver.implicitly_wait(10)

[1, 1, 260] Engineer - Kitchen Equipment 👍
[2, 1, 260] Engineering Manager 👍
[3, 1, 260] Electrical Technician 👍
[4, 1, 260] Engineering Lead 👍
[5, 1, 260] Technician 👍
[6, 1, 260] Sales Engineer 👍
[7, 1, 260] Electrician 👍
[8, 1, 260] Project Manager 👍
[9, 1, 260] Instrument & Electrical Engineer 👍
[10, 1, 260] Process Electronic Engineer 👍
[11, 1, 260] Mechanic Technician II 👍
[12, 1, 260] Duty Engineering 👍
[13, 1, 260] QA Engineer - Digital Team 👍
[14, 1, 260] Hardware Engineer 👍
[15, 1, 260] Sales Engineer 👍
[16, 1, 260] Senior Engineer - Electrical 👍
[17, 1, 260] Maintenance Technician_Surabaya 👍
[18, 1, 260] Automation and Electrician Technician 👍
[19, 1, 260] Senior Electrical Engineer 👍
[20, 1, 260] EQE Engineer 👍
[21, 1, 260] Project Manager- Electrical MV Switchgear 👍
[22, 1, 260] Electrical Project Engineer 👍
[23, 1, 260] PLANT TECHNICIAN 👍
[24, 1, 260] Quality Assurance Engineer 👍
[25, 1, 260] Engineering Technician 👍
📖 Will move to the next page (1 -> 2) from 11 total pag

In [41]:
filename = 'job_' + ''.join(job_name.title().split())

with open(f'data/scraped/{filename}.pickle', 'wb') as file:
    pickle.dump(job_results, file)

In [231]:
# with open("data/scraped/job_3DVideoAnimator.pickle", "rb") as f:
#     my_list = pickle.load(f)

# print(len(my_list))

In [232]:
# body = driver.find_element_by_tag_name("body")
# body.send_keys(Keys.CONTROL + '-')

In [233]:
# job_search = driver.find_element_by_css_selector('#global-nav-typeahead > input')

# job_search.click()
# job_search.send_keys(Keys.CONTROL + 'a')
# job_search.send_keys(Keys.DELETE)
# job_search.send_keys('ios developer')

# location_search = driver.find_element_by_css_selector('#jobs-search-box-location-id-ember1360')

# location_search.click()
# location_search.send_keys(Keys.CONTROL + 'a')
# location_search.send_keys(Keys.DELETE)
# location_search.send_keys('indonesia')

# search_button = driver.find_element_by_xpath('//*[@id="global-nav-search"]/div/div[2]/button[1]')
# search_button.click()

# actions = ActionChains(driver)
# actions.send_keys(Keys.ENTER)
# actions.perform()